## Simple XGBoost Starter (~0.0655)
date : 2020-04-09 <br>
[kaggle notebook]https://www.kaggle.com/anokas/simple-xgboost-starter-0-0655

In [101]:
import numpy as np
import pandas as pd
import xgboost as xgb
import gc

### 1. loading Data

In [102]:
train = pd.read_csv('./data/train_2016_v2.csv')
prop = pd.read_csv('./data/properties_2016.csv')
sample = pd.read_csv('./data/sample_submission.csv')

/data1/suhyuncho/.conda/envs/suhyun3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
prop.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,650756.0,1413387.0,2015.0,762631.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,NaN,...,1.0,NaN,571346.0,1156834.0,2015.0,585488.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,NaN,NaN,193796.0,433491.0,2015.0,239695.0,5725.17,NaN,NaN,NaN


In [5]:
train.head()

,parcelid,logerror,transactiondate
0,11016594,0.0276,2016-01-01
1,14366692,-0.1684,2016-01-01
2,12098116,-0.0040,2016-01-01
3,12643413,0.0218,2016-01-02
4,14432541,-0.0050,2016-01-02


In [6]:
sample.head()

,ParcelId,201610,201611,201612,201710,201711,201712
0,10754147,0,0,0,0,0,0
1,10759547,0,0,0,0,0,0
2,10843547,0,0,0,0,0,0
3,10859147,0,0,0,0,0,0
4,10879947,0,0,0,0,0,0


### 2. Binding to float32

In [17]:
# zip 은 동일한 개수로 이루어진 자료형을 묶어 주는 역할을 하는 함수
list(zip(prop.columns,prop.dtypes))[:5]

[('parcelid', dtype('int64')),
 ('airconditioningtypeid', dtype('float64')),
 ('architecturalstyletypeid', dtype('float64')),
 ('basementsqft', dtype('float64')),
 ('bathroomcnt', dtype('float64'))]

In [21]:
for c, dtype in zip(prop.columns,prop.dtypes):
    if dtype ==np.float64:
        prop[c]=prop[c].astype(np.float32)

### 3. Creating training set

In [53]:
df_train = train.merge(prop, how='left', on='parcelid')
df_train.head()

,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,11016594,0.0276,2016-01-01,1.0,NaN,NaN,2.0,3.0,NaN,4.0,...,NaN,NaN,122754.0,360170.0,2015.0,237416.0,6735.879883,NaN,NaN,6.037107e+13
1,14366692,-0.1684,2016-01-01,NaN,NaN,NaN,3.5,4.0,NaN,NaN,...,NaN,NaN,346458.0,585529.0,2015.0,239071.0,10153.019531,NaN,NaN,NaN
2,12098116,-0.0040,2016-01-01,1.0,NaN,NaN,3.0,2.0,NaN,4.0,...,NaN,NaN,61994.0,119906.0,2015.0,57912.0,11484.480469,NaN,NaN,6.037464e+13
3,12643413,0.0218,2016-01-02,1.0,NaN,NaN,2.0,2.0,NaN,4.0,...,NaN,NaN,171518.0,244880.0,2015.0,73362.0,3048.739990,NaN,NaN,6.037296e+13
4,14432541,-0.0050,2016-01-02,NaN,NaN,NaN,2.5,4.0,NaN,NaN,...,2.0,NaN,169574.0,434551.0,2015.0,264977.0,5488.959961,NaN,NaN,6.059042e+13


In [54]:
x_train = df_train.drop(['parcelid', 'logerror', 'transactiondate', 'propertyzoningdesc', 'propertycountylandusecode'], axis=1)
y_train = df_train['logerror'].values
print(x_train.shape, y_train.shape)

(90275, 55) (90275,)


In [55]:
# 데이터타입이 object인 컬럼명
x_train.dtypes[x_train.dtypes==object].index.values

array(['hashottuborspa', 'fireplaceflag', 'taxdelinquencyflag'],
      dtype=object)

In [56]:
train_columns = x_train.columns

for c in x_train.dtypes[x_train.dtypes == object].index.values:    # 데이터타입이 object인 컬럼명만 추출
    x_train[c] = (x_train[c] == True)

del df_train; gc.collect()

# 전체 90275 데이터에서 80000개는 train에 사용. 나머지는 valid에 사용.
split = 80000
x_train, y_train, x_valid, y_valid = x_train[:split], y_train[:split], x_train[split:], y_train[split:]

### Building DMatrix

In [57]:
d_train = xgb.DMatrix(x_train, label=y_train)
d_valid = xgb.DMatrix(x_valid, label=y_valid)

del x_train, x_valid; gc.collect()

117

### Training

In [66]:
params = {}
params['eta'] = 0.02
params['objective'] = 'reg:linear'
params['eval_metric'] = 'mae'
params['max_depth'] = 4
params['silent'] = 1

watchlist = [(d_train, 'train'), (d_valid, 'valid')]
clf = xgb.train(params, d_train, 10000, watchlist, early_stopping_rounds=100, verbose_eval=10)

del d_train, d_valid

[0]	train-mae:0.488065	valid-mae:0.48112
Multiple eval metrics have been passed: 'valid-mae' will be used for early stopping.

Will train until valid-mae hasn't improved in 100 rounds.
[10]	train-mae:0.402221	valid-mae:0.395444
[20]	train-mae:0.332679	valid-mae:0.326099
[30]	train-mae:0.276518	valid-mae:0.270132
[40]	train-mae:0.231316	valid-mae:0.225214
[50]	train-mae:0.195057	valid-mae:0.189328
[60]	train-mae:0.166122	valid-mae:0.160725
[70]	train-mae:0.143116	valid-mae:0.13805
[80]	train-mae:0.124973	valid-mae:0.120241
[90]	train-mae:0.110785	valid-mae:0.106352
[100]	train-mae:0.099816	valid-mae:0.095695
[110]	train-mae:0.091452	valid-mae:0.087611
[120]	train-mae:0.085136	valid-mae:0.081602
[130]	train-mae:0.080447	valid-mae:0.077224
[140]	train-mae:0.077012	valid-mae:0.074089
[150]	train-mae:0.074499	valid-mae:0.07184
[160]	train-mae:0.072682	valid-mae:0.070257
[170]	train-mae:0.071361	valid-mae:0.069146
[180]	train-mae:0.070406	valid-mae:0.068383
[190]	train-mae:0.06971	valid-mae:

### Building test set

In [71]:
sample['parcelid'] = sample['ParcelId']
df_test = sample.merge(prop, on='parcelid', how='left')
df_test

,ParcelId,201610,201611,201612,201710,201711,201712,parcelid,airconditioningtypeid,architecturalstyletypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,0,0,0,0,0,0,10754147,NaN,NaN,...,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN
1,10759547,0,0,0,0,0,0,10759547,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,0,0,0,0,0,0,10843547,NaN,NaN,...,NaN,NaN,650756.0,1413387.0,2015.0,762631.0,20800.369141,NaN,NaN,NaN
3,10859147,0,0,0,0,0,0,10859147,NaN,NaN,...,1.0,NaN,571346.0,1156834.0,2015.0,585488.0,14557.570312,NaN,NaN,NaN
4,10879947,0,0,0,0,0,0,10879947,NaN,NaN,...,NaN,NaN,193796.0,433491.0,2015.0,239695.0,5725.169922,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2985212,168176230,0,0,0,0,0,0,168176230,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2985213,14273630,0,0,0,0,0,0,14273630,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2985214,168040630,0,0,0,0,0,0,168040630,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2985215,168040830,0,0,0,0,0,0,168040830,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
x_test.head()

,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,False,NaN,9.0,2015.0,9.0,NaN,False,NaN,NaN
1,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,False,NaN,27516.0,2015.0,27516.0,NaN,False,NaN,NaN
2,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,False,650756.0,1413387.0,2015.0,762631.0,20800.369141,False,NaN,NaN
3,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,NaN,NaN,...,1.0,False,571346.0,1156834.0,2015.0,585488.0,14557.570312,False,NaN,NaN
4,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,NaN,...,NaN,False,193796.0,433491.0,2015.0,239695.0,5725.169922,False,NaN,NaN


In [83]:
sample['parcelid'] = sample['ParcelId']
df_test = sample.merge(prop, on='parcelid', how='left')

del prop; gc.collect()

x_test = df_test[train_columns]
for c in x_test.dtypes[x_test.dtypes == object].index.values:
    x_test[c] = (x_test[c] == True)

del df_test, sample; gc.collect()

d_test = xgb.DMatrix(x_test)

del x_test; gc.collect()

/data1/suhyuncho/.conda/envs/suhyun3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


193

### Predicting on test

In [87]:
# clf는 train시킨것 
p_test = clf.predict(d_test)
p_test

array([ 0.19058421, -0.02764982,  0.09469894, ...,  0.27551016,
        0.27551016,  0.27551016], dtype=float32)

In [88]:
sub = pd.read_csv('./data/sample_submission.csv')
sub

,ParcelId,201610,201611,201612,201710,201711,201712
0,10754147,0,0,0,0,0,0
1,10759547,0,0,0,0,0,0
2,10843547,0,0,0,0,0,0
3,10859147,0,0,0,0,0,0
4,10879947,0,0,0,0,0,0
...,...,...,...,...,...,...,...
2985212,168176230,0,0,0,0,0,0
2985213,14273630,0,0,0,0,0,0
2985214,168040630,0,0,0,0,0,0
2985215,168040830,0,0,0,0,0,0


In [92]:
sub.columns[sub.columns!='ParcelId']

Index(['201610', '201611', '201612', '201710', '201711', '201712'], dtype='object')

In [93]:
for c in sub.columns[sub.columns!='ParcleId']:
    sub[c]=p_test

In [94]:
sub

,ParcelId,201610,201611,201612,201710,201711,201712
0,0.190584,0.190584,0.190584,0.190584,0.190584,0.190584,0.190584
1,-0.027650,-0.027650,-0.027650,-0.027650,-0.027650,-0.027650,-0.027650
2,0.094699,0.094699,0.094699,0.094699,0.094699,0.094699,0.094699
3,0.316670,0.316670,0.316670,0.316670,0.316670,0.316670,0.316670
4,0.037622,0.037622,0.037622,0.037622,0.037622,0.037622,0.037622
...,...,...,...,...,...,...,...
2985212,0.275510,0.275510,0.275510,0.275510,0.275510,0.275510,0.275510
2985213,0.275510,0.275510,0.275510,0.275510,0.275510,0.275510,0.275510
2985214,0.275510,0.275510,0.275510,0.275510,0.275510,0.275510,0.275510
2985215,0.275510,0.275510,0.275510,0.275510,0.275510,0.275510,0.275510


In [ ]:
# clf = xgb.train(params, d_train, 10000, watchlist, early_stopping_rounds=100, verbose_eval=10)
p_test = clf.predict(d_test)

del d_test; gc.collect()

sub = pd.read_csv('./data/sample_submission.csv')
for c in sub.columns[sub.columns != 'ParcelId']:
    sub[c] = p_test

### Writing csv 

In [98]:
sub.to_csv('xgb_starter.csv',index=False,float_format='%.4f', header =1)

### 원본 코드

In [105]:
train = pd.read_csv('./data/train_2016_v2.csv')
prop = pd.read_csv('./data/properties_2016.csv')
sample = pd.read_csv('./data/sample_submission.csv')

print('Binding to float32')

for c, dtype in zip(prop.columns, prop.dtypes):
	if dtype == np.float64:
		prop[c] = prop[c].astype(np.float32)

print('Creating training set ...')

df_train = train.merge(prop, how='left', on='parcelid')

x_train = df_train.drop(['parcelid', 'logerror', 'transactiondate', 'propertyzoningdesc', 'propertycountylandusecode'], axis=1)
y_train = df_train['logerror'].values
print(x_train.shape, y_train.shape)

train_columns = x_train.columns

for c in x_train.dtypes[x_train.dtypes == object].index.values:
    x_train[c] = (x_train[c] == True)

del df_train; gc.collect()

split = 80000
x_train, y_train, x_valid, y_valid = x_train[:split], y_train[:split], x_train[split:], y_train[split:]

print('Building DMatrix...')

d_train = xgb.DMatrix(x_train, label=y_train)
d_valid = xgb.DMatrix(x_valid, label=y_valid)

del x_train, x_valid; gc.collect()

print('Training ...')

params = {}
params['eta'] = 0.02
params['objective'] = 'reg:linear'
params['eval_metric'] = 'mae'
params['max_depth'] = 4
params['silent'] = 1

watchlist = [(d_train, 'train'), (d_valid, 'valid')]
clf = xgb.train(params, d_train, 10000, watchlist, early_stopping_rounds=100, verbose_eval=10)

del d_train, d_valid

print('Building test set ...')

sample['parcelid'] = sample['ParcelId']
df_test = sample.merge(prop, on='parcelid', how='left')

del prop; gc.collect()

x_test = df_test[train_columns]
for c in x_test.dtypes[x_test.dtypes == object].index.values:
    x_test[c] = (x_test[c] == True)

del df_test, sample; gc.collect()

d_test = xgb.DMatrix(x_test)

del x_test; gc.collect()

print('Predicting on test ...')

p_test = clf.predict(d_test)

del d_test; gc.collect()

sub = pd.read_csv('./data/sample_submission.csv')
for c in sub.columns[sub.columns != 'ParcelId']:
    sub[c] = p_test

print('Writing csv ...')
sub.to_csv('xgb_starter.csv', index=False, float_format='%.4f') 

Binding to float32
Creating training set ...
(90275, 55) (90275,)
Building DMatrix...
Training ...
[0]	train-mae:0.488065	valid-mae:0.48112
Multiple eval metrics have been passed: 'valid-mae' will be used for early stopping.

Will train until valid-mae hasn't improved in 100 rounds.
[10]	train-mae:0.402221	valid-mae:0.395444
[20]	train-mae:0.332679	valid-mae:0.326099
[30]	train-mae:0.276518	valid-mae:0.270132
[40]	train-mae:0.231316	valid-mae:0.225214
[50]	train-mae:0.195057	valid-mae:0.189328
[60]	train-mae:0.166122	valid-mae:0.160725
[70]	train-mae:0.143116	valid-mae:0.13805
[80]	train-mae:0.124973	valid-mae:0.120241
[90]	train-mae:0.110785	valid-mae:0.106352
[100]	train-mae:0.099816	valid-mae:0.095695
[110]	train-mae:0.091452	valid-mae:0.087611
[120]	train-mae:0.085136	valid-mae:0.081602
[130]	train-mae:0.080447	valid-mae:0.077224
[140]	train-mae:0.077012	valid-mae:0.074089
[150]	train-mae:0.074499	valid-mae:0.07184
[160]	train-mae:0.072682	valid-mae:0.070257
[170]	train-mae:0.07136

/data1/suhyuncho/.conda/envs/suhyun3/lib/python3.7/site-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Predicting on test ...
Writing csv ...
